# SQL I

Introducing SQL and databases.

## Starting Up SQL

Before we look at SQL syntax in detail, let's first get ourselves set up to run SQL queries in Jupyter.

#### Approach #1: SQL Magic

**1. Load the `sql` Module.** 

Load `%%sql` cell magic.

In [1]:
!pip install sql

In [7]:
load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


**2. Connect to a database.**  

Here, we connect to the SQLite database `basic_examples.db`.

In [10]:
%%sql
sqlite:///basic_examples.db

<br/>

**3. Run a simple SQL query.** 

Note the `%%sql` lets Jupyter parse the rest of the lines as a SQL command.

In [9]:
%%sql
select * FROM Dragon;

 * sqlite:///basic_examples.db
Done.


name,year,cute
hiccup,2010,10
drogon,2011,-100
dragon 2,2019,0
puff,2010,100
smaug,2011,None


**Simple query, this time on two different lines.**

In [6]:
%%sql
SELECT *
FROM Dragon;

 * sqlite:///basic_examples.db
Done.


name,year,cute
hiccup,2010,10
drogon,2011,-100
dragon 2,2019,0
puff,2010,100
smaug,2011,None


#### Approach #2: `pd.read_sql`

It turns out that `pandas` has a special-purpose function to parse SQL queries. We can pass in a SQL query as a string to return a `pandas` DataFrame. To achieve the same result as we did using cell magic above, we can do the following.

**1. Connect to a database**

In [9]:
import sqlalchemy 
import pandas as pd

engine = sqlalchemy.create_engine("sqlite:///basic_examples.db")
connection = engine.connect()

**2. Run a simple SQL query**

In [10]:
query = """
SELECT * 
FROM Dragon;
"""

pd.read_sql(query, engine)

,name,year,cute
0,hiccup,2010,10.0
1,drogon,2011,-100.0
2,dragon 2,2019,0.0
3,puff,2010,100.0
4,smaug,2011,NaN


## Tables and Schema

A **database** contains a collection of SQL **tables**. Let's connect to our "toy" database `basic_examples.db` and explore the tables it stores.

In [11]:
%%sql
sqlite:///basic_examples.db

In [13]:
%%sql
SELECT * FROM sqlite_master WHERE type='table'

 * sqlite:///basic_examples.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,7,"CREATE TABLE sqlite_sequence(name,seq)"
table,Dragon,Dragon,2,"CREATE TABLE Dragon ( name TEXT PRIMARY KEY, year INTEGER CHECK (year >= 2000), cute INTEGER)"
table,Dish,Dish,4,"CREATE TABLE Dish ( name TEXT PRIMARY KEY, type TEXT, cost INTEGER CHECK (cost >= 0))"
table,Scene,Scene,6,"CREATE TABLE Scene ( id INTEGER PRIMARY KEY AUTOINCREMENT, biome TEXT NOT NULL, city TEXT NOT NULL, visitors INTEGER CHECK (visitors >= 0), created_at DATETIME DEFAULT (DATETIME('now')))"


## Basic Queries

Every SQL query *must* contain a `SELECT` and `FROM` clause.

* `SELECT`: specify the column(s) to return in the output
* `FROM`: specify the database table from which to extract data

**Question:** Select all columns from the **Dragon** table.

In [16]:
query='''select * from Dragon'''
pd.read_sql(query,engine)

,name,year,cute
0,hiccup,2010,10.0
1,drogon,2011,-100.0
2,dragon 2,2019,0.0
3,puff,2010,100.0
4,smaug,2011,NaN


**Question:** Select columns **cute** and **year** from the **Dragon** table.

In [17]:
query='''select cute,year from Dragon'''
pd.read_sql(query,engine)

,cute,year
0,10.0,2010
1,-100.0,2011
2,0.0,2019
3,100.0,2010
4,NaN,2011


**Aliasing** with `AS`

**Question:** Repeat the last exercise with aliasing.

In [18]:
query='''select cute as CUTE_2,year as YEAR_1 from Dragon'''
pd.read_sql(query,engine)

,CUTE_2,YEAR_1
0,10.0,2010
1,-100.0,2011
2,0.0,2019
3,100.0,2010
4,NaN,2011


**Uniqueness** with `DISTINCT`

**Question:** Select all the unique years in the **Dragon** table.

In [21]:
query='''select distinct Year from Dragon'''
pd.read_sql(query,engine)

,year
0,2010
1,2011
2,2019


**Filtering** with `WHERE`

**Question:** Select the **name** and **year** columns from the **Dragon** table such that the cute value is greater than 0.

In [23]:
query='''select name,year from Dragon where cute>0'''
pd.read_sql(query,engine)

,name,year
0,hiccup,2010
1,puff,2010


**Question:** Select the **name**, **cute** and **year** columns from the **Dragon** table such that the cute value is greater than 0 or the year is greater than 2013.

In [24]:
query='''select * from Dragon where cute>0 or year>2013'''
pd.read_sql(query,engine)

,name,year,cute
0,hiccup,2010,10
1,dragon 2,2019,0
2,puff,2010,100


**Question:** Select the **name** and **year** columns from the **Dragon** table such that the name is either 'puff' or 'hiccup'.

In [27]:
query='''select name,year from Dragon where name='puff' or name='hiccup' '''
pd.read_sql(query,engine)

,name,year
0,hiccup,2010
1,puff,2010


**Question:** Get the name and cute value of all dragons whose cute value is not null.

In [28]:
query='''select name,cute from Dragon where cute is not null'''
pd.read_sql(query,engine)

,name,cute
0,hiccup,10
1,drogon,-100
2,dragon 2,0
3,puff,100


**Ordering** data using `ORDER BY`

**Question:** Sort the **Dragon** table in descending order of cuteness.

In [30]:
query='''select * from Dragon order by cute DESC'''
pd.read_sql(query,engine)

,name,year,cute
0,puff,2010,100.0
1,hiccup,2010,10.0
2,dragon 2,2019,0.0
3,drogon,2011,-100.0
4,smaug,2011,NaN


**Restricting** output with `LIMIT` and `OFFSET`

**Question:** Query the first two rows of the **Dragon** table.

In [39]:
query='''select * from Dragon limit 2 '''


pd.read_sql(query,engine)

,name,year,cute
0,hiccup,2010,10
1,drogon,2011,-100


**Question:** Query the two rows after the first row of the **Dragon** table. 

In [44]:
query='''select * from Dragon limit 3 offset 2 '''


pd.read_sql(query,engine)

,name,year,cute
0,dragon 2,2019,0.0
1,puff,2010,100.0
2,smaug,2011,NaN


## Grouping Data with `GROUP BY`

**Question:** Get all rows and columns of the **Dish** table. 

In [46]:
query='''select * from DISH '''


pd.read_sql(query,engine)

,name,type,cost
0,ravioli,entree,10
1,ramen,entree,13
2,taco,entree,7
3,edamame,appetizer,4
4,fries,appetizer,4
5,potsticker,appetizer,4
6,ice cream,dessert,5


A small note: the fact that `type` is highlighted in green below is a consequence of Jupyter assuming that we are writing Python code (where `type` is a built-in keyword). `type` does *not* have a special meaning in SQL, so the color below does not indicate any special functionality. When we run the cell, Jupyter realizes it should recognize the code as SQL. 

**Question:** Select the **type** column of the **Dish** table. 

In [47]:
query='''select type from Dish'''


pd.read_sql(query,engine)

,type
0,entree
1,entree
2,entree
3,appetizer
4,appetizer
5,appetizer
6,dessert


**Question:** Get all the dish types using GROUP BY. 

In [49]:
query='''select type from Dish group by type'''

pd.read_sql(query,engine)

,type
0,appetizer
1,dessert
2,entree


**Question:** Query the total cost of each type of dish. 

In [54]:
query='''select type,sum(cost) from Dish group by type '''

pd.read_sql(query,engine)

,type,sum(cost)
0,appetizer,12
1,dessert,5
2,entree,30


**Question:** Query the total cost, the minimum cost and the name of the most expensive dish of each type. 

In [56]:
query='''select type,sum(cost),min(cost),max(cost),max(name) from Dish group by type '''

pd.read_sql(query,engine)

,type,sum(cost),min(cost),max(cost),max(name)
0,appetizer,12,4,4,potsticker
1,dessert,5,5,5,ice cream
2,entree,30,7,13,taco


**Question:** Count the number of rows in each year in the **Dragon** table. 

In [68]:
query=''' select year,count(*) from dragon group by year '''

pd.read_sql(query,engine)

,year,count(*)
0,2010,2
1,2011,2
2,2019,1


**Question:** Count the number of rows - including the rows with NULLs - in each year in the **Dragon** table. 

In [67]:
query=''' select year,count(year) from dragon group by year '''

pd.read_sql(query,engine)

,year,count(year)
0,2010,2
1,2011,2
2,2019,1
